# Imports

In [1]:
import MySQLdb
from pymongo import MongoClient
import pandas
import numpy as np
import json

# Conexão com a base no MongoDB

In [2]:
# Conexão com o MongoDB
client = MongoClient()
client = MongoClient('localhost', 27017)

# Cria uma base de dados no MongoDB
db_mongo = client['tf_f_dupla1_fim']

# Conexão com a base no MySQL

In [3]:
# Conexão com o MySQL
db = MySQLdb.connect("localhost","root","SENHA","tf_f_dupla1_fim")
cursor = db.cursor()

# Configurações do MySQL
db.set_character_set('utf8')
cursor.execute('SET NAMES utf8;')
cursor.execute('SET CHARACTER SET utf8;')
cursor.execute('SET character_set_connection=utf8;')

0

# Classe para ler e salvar os 'Credits' nas bases

In [4]:
class ReadCredit():
    def __init__(self, path):
        self.data = None
        self.json_data = None
        self.path = path
        self.insert = """INSERT INTO CREDIT VALUES (%s, %s, %s)"""
        
        self.read_data()


    def read_data(self):
        self.data = pandas.read_csv(self.path,
                        usecols=['id', 'cast', 'crew'])
        self.data = self.data.replace(np.nan, None)
        self.json_data = self.data.to_json(orient = "records")


    def save_data(self):
        for idx, mydata in self.data.iterrows():
            idMovie = mydata['id']
            cast = json.dumps(mydata['cast'], ensure_ascii=False).encode('utf8')
            crew = json.dumps(mydata['crew'], ensure_ascii=False).encode('utf8')
            
            ans = (idMovie, cast, crew)
            try:
                cursor.execute(self.insert, ans)
            except Exception as e:
                 print(e)

        db.commit()


    def save_mongo(self):
        json_credits = json.loads(self.json_data)

        credits = db_mongo['credit']
        credits.insert_many(json_credits)

# Função principal

In [5]:
def main():
    database_save = ReadCredit('./credits.csv')
    database_save.save_data()
    database_save.save_mongo()
    db.close()

In [ ]:
main()